<a href="https://colab.research.google.com/github/elylaila/seminario/blob/main/Explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gsarti/lcl23-xnlm-lab/blob/main/notebooks/2_Attribution_Contrastive.ipynb)

In [1]:
%%capture
# Run in Colab to install local packages
%pip install ferret-xai bitsandbytes accelerate numpy==1.23.1
%pip install git+https://github.com/huggingface/transformers.git
# %pip install git+https://github.com/inseq-team/inseq.git

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pandas as pd
from ferret import Benchmark

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Seminario/

Mounted at /content/drive


In [20]:
name = 'dbmdz/bert-base-italian-cased' # inserire bert-ita
model = AutoModelForSequenceClassification.from_pretrained('final_model/checkpoint-4375') # inserire path modello finale
tokenizer = AutoTokenizer.from_pretrained(name)
print(model.config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1'}


In [15]:
predictions = pd.read_csv('final_model_predictions.csv', usecols=['y_pred', 'y_true'])
predictions

,y_true,y_pred
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
995,0,0
996,0,1
997,0,0
998,1,1


In [87]:
df = pd.read_csv('dataset/HODI_2023_test_GOLD.tsv', sep='\t').drop(columns=['homotransphobic'])
df = pd.concat([df, predictions], axis=1)
df

,id,text,rationales,y_true,y_pred
0,5001,@user_abcdefg @user_abc Quasi quasi è meglio f...,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",1,0
1,5002,"@user_abcdefgh Sotto quel tweet, c'è un ammass...",NaN,0,0
2,5003,@user_a brava Mara come sempre puntatona.. Sei...,NaN,0,0
3,5004,copio i gay cagatemi !!! se vi chiedessi di de...,NaN,0,0
4,5005,io: è bono eh papà: ma non eri frocia te 🤡🤡🤡,NaN,0,0
...,...,...,...,...,...
995,5996,RT @user_abcdefghij @user_abcdefghij Quindi tu...,NaN,0,0
996,5997,@user_abcde Quel “sono a lavoro André” che suo...,NaN,0,1
997,5998,@user_abcdefghij @user_a 115 gol e 48 assist i...,NaN,0,0
998,5999,@user_abcdefgh Ma non rompermi le balle culatt...,"[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5...",1,1


In [88]:
df_errors_01 = df[(df['y_true']==0) & (df['y_pred']==1)].drop(columns=['rationales'])
df_errors_01.shape

(53, 4)

In [89]:
df_errors_01_sample = df_errors_01.sample(n=5, random_state=42)
df_errors_01_sample

,id,text,y_true,y_pred
383,5384,@user_abcdef @user_abcdef 4) se un ragazzo des...,0,1
817,5818,comunque siamo 5 froci in una piccola macchi a...,0,1
911,5912,"AUG 19 2022, 19:36: BERNARDINO FEMMINIELLI [URL]",0,1
292,5293,@user_abcdefg @user_abcdefgh @user_abcde Io pr...,0,1
853,5854,"@user_abcd Pensa , prima di diceva ricchione...",0,1


In [90]:
sentences = list(df_errors_01_sample.text)
sentences

['@user_abcdef @user_abcdef 4) se un ragazzo desse un pugno in faccia ad una ragazza perché ha insultato uno con un "checca di merda" avrebbe non solo il mio appoggio ma anche la mia ammirazione.',
 'comunque siamo 5 froci in una piccola macchi a diretti verso una serata frocia',
 'AUG 19 2022, 19:36: BERNARDINO FEMMINIELLI [URL]',
 '@user_abcdefg @user_abcdefgh @user_abcde Io preferisco fidarmi da chi non nasconde il suo volto ma come sempre la gente che pensa di fare il “culattone” col le terga dell’altro abbonda ¯\\_(ツ)_/¯',
 '@user_abcd Pensa , prima di diceva ricchione...']

In [32]:
bench = Benchmark(model, tokenizer)
all_explanations = bench.explain(sentences[0], target=1)
bench.show_table(all_explanations)

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

,4,),se,un,ragazzo,des,##se,un.1,pugno,in,faccia,ad,una,ragazza,perché,ha,insul,##tato,uno,con,un.2,"""",che,##cca,di,merda,""".1",avrebbe,non,solo,il,mio,appoggio,ma,anche,la,mia,ammira,##zione,.
Partition SHAP,-0.10,-0.17,-0.01,0.01,0.01,-0.00,-0.02,0.00,-0.04,-0.04,-0.03,-0.00,-0.00,0.00,0.01,0.02,-0.02,-0.02,0.01,-0.00,0.07,0.02,0.12,0.05,0.04,0.05,-0.02,-0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.00,-0.01,-0.01,-0.05,0.01
LIME,-0.01,-0.01,-0.00,0.01,-0.02,-0.03,0.01,0.01,-0.05,-0.02,-0.01,0.00,0.01,-0.05,0.00,0.01,-0.09,0.00,0.01,-0.02,0.06,0.00,0.13,0.06,0.05,0.07,-0.14,-0.02,0.01,0.01,0.01,-0.02,0.02,-0.00,-0.00,0.01,-0.00,0.01,0.00,-0.01
Gradient,0.03,0.02,0.01,0.01,0.02,0.02,0.01,0.02,0.05,0.02,0.02,0.01,0.01,0.03,0.02,0.02,0.06,0.03,0.02,0.02,0.03,0.05,0.03,0.05,0.03,0.06,0.04,0.02,0.01,0.01,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.01,0.04
Gradient (x Input),0.01,0.00,0.03,0.01,0.02,-0.01,0.01,0.01,-0.02,-0.02,-0.04,-0.02,-0.01,-0.01,-0.02,-0.03,-0.05,0.07,0.04,0.02,0.01,-0.05,0.04,-0.01,-0.10,-0.02,-0.03,-0.01,0.02,-0.02,-0.03,-0.01,0.01,0.02,0.02,-0.02,-0.02,0.01,0.00,-0.09
Integrated Gradient,-0.01,-0.01,0.00,-0.03,-0.06,0.03,0.04,0.09,-0.06,-0.02,0.00,-0.01,-0.01,-0.00,-0.01,0.01,-0.01,0.02,-0.03,-0.03,0.01,0.04,0.09,0.04,-0.03,0.05,-0.06,0.03,-0.01,0.01,-0.00,-0.00,-0.00,-0.01,0.00,-0.00,-0.01,0.04,0.01,-0.02
Integrated Gradient (x Input),0.01,-0.00,-0.00,0.02,0.04,-0.04,-0.02,-0.02,-0.10,-0.02,-0.03,-0.00,-0.01,-0.04,-0.02,0.01,-0.09,0.04,0.01,-0.00,0.03,0.00,0.12,0.12,-0.00,0.08,-0.09,-0.01,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.01,-0.00,-0.01,0.00,0.00


In [40]:
sentences[1]

'comunque siamo 5 froci in una piccola macchi a diretti verso una serata frocia'

In [33]:
all_explanations = bench.explain(sentences[1], target=1)
bench.show_table(all_explanations)

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

,comunque,siamo,5,fro,##ci,in,una,piccola,macch,##i,a,diretti,verso,una.1,serata,fro.1,##cia
Partition SHAP,-0.02,0.06,0.09,0.26,0.01,0.01,-0.05,-0.10,0.03,-0.03,-0.01,0.02,0.02,-0.03,-0.13,-0.06,-0.07
LIME,0.02,0.01,0.05,0.46,0.05,-0.00,-0.03,-0.06,0.06,-0.01,0.02,0.02,0.02,0.03,-0.08,0.01,-0.07
Gradient,0.05,0.05,0.06,0.14,0.07,0.03,0.03,0.04,0.08,0.03,0.02,0.05,0.03,0.03,0.08,0.04,0.07
Gradient (x Input),-0.15,-0.03,-0.07,-0.15,0.03,-0.05,0.00,0.00,-0.08,-0.03,-0.04,0.07,-0.01,-0.01,0.03,0.03,-0.13
Integrated Gradient,-0.01,0.03,0.02,-0.22,0.02,0.02,0.02,-0.03,0.09,-0.08,0.00,0.09,-0.01,0.09,-0.18,-0.01,-0.02
Integrated Gradient (x Input),0.06,0.03,-0.06,0.23,0.11,-0.07,-0.08,-0.03,0.02,-0.01,-0.02,-0.00,0.02,-0.05,-0.11,-0.09,-0.02


In [39]:
sentences[2]

'aug 19 2022, 19:36: bernardino femminielli'

In [34]:
all_explanations = bench.explain(sentences[2], target=1)
bench.show_table(all_explanations)

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

,au,##g,19,202,##2,",",19.1,:,36,:.1,ber,##nar,##dino,femmini,##elli
Partition SHAP,-0.12,0.12,-0.05,0.04,0.01,-0.00,-0.01,-0.10,0.07,-0.11,0.03,0.11,0.06,-0.11,0.04
LIME,-0.05,-0.01,-0.06,0.03,-0.01,-0.02,-0.01,-0.09,0.00,-0.20,-0.11,0.12,0.07,-0.13,0.10
Gradient,0.04,0.04,0.03,0.05,0.03,0.03,0.04,0.05,0.04,0.08,0.06,0.07,0.09,0.13,0.08
Gradient (x Input),-0.04,-0.14,0.01,0.00,-0.01,0.01,-0.04,0.07,0.03,-0.23,-0.09,0.00,0.12,-0.04,-0.08
Integrated Gradient,-0.00,0.04,0.05,0.01,-0.02,0.00,0.03,-0.09,-0.01,-0.15,-0.06,0.11,-0.15,-0.10,0.03
Integrated Gradient (x Input),-0.09,0.02,0.01,0.00,-0.06,0.00,-0.01,-0.06,0.08,-0.10,-0.11,-0.07,0.24,-0.09,0.05


In [41]:
all_explanations = bench.explain(sentences[3], target=1)
bench.show_table(all_explanations)

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [91]:
df_correct_11 = df[(df['y_true']==1) & (df['y_pred']==1)]
df_correct_11.shape

(348, 5)

In [92]:
df_correct_11_sample = df_correct_11.sample(n=5, random_state=42)
df_correct_11_sample

,id,text,rationales,y_true,y_pred
741,5742,@user_abcdefghi ma sei rotto in culo tu che ca...,"[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2...",1,1
359,5360,@user_abcdef Certo è un noto culattone @user_a...,"[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 2...",1,1
892,5893,@user_abcd CHI ? AHHH QUESTO COGLIONE RICCHION...,"[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3...",1,1
776,5777,@user_abcdefgh Questi hanno tutta la mia stima...,"[74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 8...",1,1
509,5510,@user_abcdefg Sai sparare ma sei rotto in culo,"[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4...",1,1


In [44]:
sentences_11 = list(df_correct_11_sample.text)
all_explanations = bench.explain(sentences_11[0], target=1)
bench.show_table(all_explanations)

Explainer:   0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [70]:
def get_indexes(frase):


In [93]:
def get_rationales(rationales_str, frase):

  lst = rationales_str[1:-1].split(', ')
  rationales_lst = [int(e) for e in lst]

  # print(rationales_lst)

  rationales_ch = ''

  for ch_idx in rationales_lst:
    rationales_ch = rationales_ch + frase[ch_idx]

  return rationales_ch

In [86]:
rationales_str = df_correct_11_sample['rationales'].iloc[1]
frase = df_correct_11_sample['text'].iloc[1]
len(frase), rationales_str

(82,
 '[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]')

In [ ]:
rationales_dict[frase] = get_rationales(rationales_str, frase)

In [94]:
rationales_dict = {}

for i in range(5):
  rationales_str = df_correct_11_sample['rationales'].iloc[i]
  frase = df_correct_11_sample['text'].iloc[i]
  rationales_dict[frase] = get_rationales(rationales_str, frase)

In [95]:
rationales_dict

{'@user_abcdefghi ma sei rotto in culo tu che cazzo significa': 'ma sei rotto in culo tu',
 '@user_abcdef Certo è un noto culattone @user_abcdefghij la carriera l‘ha fatta con Arpa testa bassa e culo alzato': 'Certo è un noto culattone @user_abcdefghijtesta bassa e culo alzato',
 '@user_abcd CHI ? AHHH QUESTO COGLIONE RICCHIONE SODOMITA PEDOFILO… 😂… HA ANCHE LA TESTA A FORMA DI GLANDE. STAI ATTENTO PELATO, CHE SIAMO IN TANTI A PISCIARTI IN TESTA, TI RICRESCONO I CAPELLI POI… NON ABBOCCATE SONO BALLE. IL PRIMO SCHIAFFO LO PRENDE LUI. 🎪🤡🤡🤡': 'QUESTO COGLIONE RICCHIONE SODOMITA PEDOFILO… 😂… HA ANCHE LA TESTA A FORMA DI GLANDEIL PRIMO SCHIAFFO LO PRENDE LUI. 🎪🤡🤡🤡',
 "@user_abcdefgh Questi hanno tutta la mia stima perché hanno fatto il culo ai nazi rottinculo di Azov. Per i quali le nullità in servizio permanente come Gramellini trovavano le giustificazioni tipiche della merda che elargiscono con cura maniacale. Attaccati fortissimo al caxxo! Per l'eternità! [URL]": 'ai nazi rottinculo di A

In [75]:
rationales_str = df_correct_11_sample['rationales'].iloc[0]
rationales_lst = get_indexes(rationales_str)
frase = df_correct_11_sample['text'].iloc[0]

In [77]:
frase

'ma sei rotto in culo tu che cazzo significa'

In [76]:
rationales_ch = ''

for ch_idx in rationales_lst:
  rationales_ch = rationales_ch + frase[ch_idx]

print(rationales_ch)

culo tu che cazzo signi


In [37]:
df_errors_10 = df[(df['y_true']==1) & (df['y_pred']==0)].drop(columns=['rationales'])
df_errors_10.shape

(163, 3)

In [ ]:
df_errors_10_sample = df_errors_10.sample(n=5, random_state=42)
df_errors_10_sample